In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.8


In [2]:
%%time
from azureml.core.model import Model
from classifier.create_ws import AZHelper
import os

ws = AZHelper.load_ws()
model=Model(ws, 'xception_wf_1')
print(model.name, model.id, model.version, sep = '\t')

Found the config file in: /home/aaditya/PycharmProjects/Codefundopp/classifier/aml_config/config.json


xception_wf_1	xception_wf_1:2	2
CPU times: user 213 ms, sys: 5.94 ms, total: 219 ms
Wall time: 2.74 s


In [3]:
# test(xp, test_dl)




from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'torch', 'torchvision'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.8
  - torch
  - torchvision



In [3]:
import shutil

shutil.copy('classifier/scoring_on_azure.py', 'scoring_on_azure.py')

'scoring_on_azure.py'

import shutil

shutil.copy('classifier/scoring_on_azure.py', 'scoring_on_azure.py')

In [4]:
from azureml.core import Image
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script='scoring_on_azure.py', 
                                                  runtime='python', 
                                                  conda_file='myenv.yml',
                                                  description='xception wildfire image')


# create the image
image = Image.create(name='xception-wf', models=[model], image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)
# image = Image(ws, name='xception-wf', version='5')


Creating image


Running

In [5]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=2, 
                                               tags={'data': 'wildfire',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='Classify wildfire/not_wildfire using transfer learning with PyTorch')

In [6]:
%%time
from azureml.core.webservice import Webservice

service_name = 'aci-wildfire'
service = Webservice.deploy_from_image(workspace=ws,
                                       name=service_name,
                                       image=image,
                                       deployment_config=aciconfig,)

service.wait_for_deployment(show_output=True)
print(service.state)

Creating service


Running

.

.

.

...

.

.
Failed

ACI service creation operation finished, operation "Failed"


Service creation polling reached terminal state, current service state: Unhealthy
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance aci-wildfire.\nYou can also try to run image pyt1acritngjsah.azurecr.io/xception-wf:6 locally. Please refer to http://aka.ms/debugimage for more information."
    }
  ]
}


Unhealthy
CPU times: user 2.43 s, sys: 119 ms, total: 2.55 s
Wall time: 3min 37s


In [13]:
print(service.scoring_uri)

http://104.45.178.57:80/score


In [9]:
print(service.get_logs())

2019-01-21T19:50:15,011427610+00:00 - nginx/run 
2019-01-21T19:50:15,012221114+00:00 - iot-server/run 
ok: run: gunicorn: (pid 12) 0s
ok: run: rsyslog: (pid 14) 0s
ok: run: nginx: (pid 11) 0s
ok: run: rsyslog: (pid 14) 0s
2019-01-21T19:50:15,018739742+00:00 - gunicorn/run 
ok: run: rsyslog: (pid 14) 0s
2019-01-21T19:50:15,040396537+00:00 - rsyslog/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-01-21T19:50:15,159009855+00:00 - iot-server/finish 1 0
2019-01-21T19:50:15,159991559+00:00 - Exit code 1 is normal. Not restarting iot-server.
{"timestamp": "2019-01-21T19:50:15.379525Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-711866e6262844688b9f7374690d3618-cf85ff94e073dc94d8b912", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-01-21T19:

In [14]:
service.delete()

In [7]:
print(Model.get_model_path(model_name='xception_wf_1', _workspace=ws))

AzureMLException: No files to download. Did you upload files?

In [12]:
service._webservice_type

'ACI'